In [2]:
import pandas as pd
import os

df = pd.read_csv('train.csv', delimiter='|')

In [ ]:
import TTS

In [3]:
df

,wav_filename,wav_filesize,transcript,transcript_wav2vec,levenshtein,duration,num_words,client_id
0,train/audio/1840/6610/1840_6610_004677.wav,693646,Madame de Maufrigneuse était à demi pensive: m...,madame de maufrigneuse était à demi pensive mê...,0.945000,15.727937,28,1840
1,train/audio/1840/5824/1840_5824_000382.wav,567404,Sylvie habitait le deuxième étage de sa maison...,sylvie habitait le deuxième étage de sa maison...,0.973958,12.865306,31,1840
2,train/audio/1840/974/1840_974_001233.wav,578924,On pouvait espérer qu'en choisissant un terrai...,on pouvait espérer qu'en choisissant un terrai...,0.994253,13.126531,25,1840
3,train/audio/9804/10527/9804_10527_000300.wav,687886,"Vous-même, affublé d'une fausse barbe et d'une...",vous-même affaiblez dune fausse barbe et dune ...,0.968254,15.597324,32,9804
4,train/audio/1840/974/1840_974_001486.wav,621164,"Sur terre, en effet, ces animaux sont lourds, ...",sur terre en effet ces animaux sont lourds peu...,0.915152,14.084354,27,1840
...,...,...,...,...,...,...,...,...
107593,train/audio/1840/2830/1840_2830_002954-0005.wav,130604,Il en faisait une noce à ce moment-là!,il en faisait une noce à ce moment-là,1.000000,2.960544,7,1840
107594,train/audio/1840/2830/1840_2830_002954-0006.wav,69164,Mais ce n'est plus ça;,mais ce n'est plus ça,1.000000,1.567347,4,1840
107595,train/audio/1840/2830/1840_2830_002954-0007.wav,121004,il ne lui fait pas de queues,il ne lui fait pas de queue,0.964286,2.742857,6,1840
107596,train/audio/1840/4141/1840_4141_003614-0001.wav,238124,"Comme une bête sauvage blessée, elle aimait la...",comme une bête sauvage blessée elle aimeit la ...,0.981481,5.398639,8,1840


In [1]:
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.tacotron2_config import Tacotron2Config

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.utils.speakers import SpeakerManager

/home/ihernandez/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ihernandez/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:59: UserWarning: Pandas requires version '1.3.2' or newer of 'bottleneck' (version '1.2.1' currently installed).
  from pandas.core import (


In [2]:
!pip install pandas

/var/lib/oar/.batch_job_bashrc: line 5: /home/ihernandez/.bashrc: No such file or directory
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from TTS.tts.configs.shared_configs import BaseDatasetConfig
import os

output_path = "/srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/french/cml_tts_dataset_french_v0.1"
if not os.path.exists(output_path):
    os.makedirs(output_path)

dataset_config= BaseDatasetConfig(formatter='ljspeech', dataset_name='', path=output_path, meta_file_train='train.csv',
                                  ignored_speakers=None, language='fr', phonemizer='espeak', meta_file_val='', meta_file_attn_mask='')

audio_config = BaseAudioConfig(sample_rate=24000, resample=True, do_trim_silence=True, trim_db=23.0)

In [11]:
print(audio_config)

BaseAudioConfig(fft_size=1024, win_length=1024, hop_length=256, frame_shift_ms=None, frame_length_ms=None, stft_pad_mode='reflect', sample_rate=24000, resample=True, preemphasis=0.0, ref_level_db=20, do_sound_norm=False, log_func='np.log10', do_trim_silence=True, trim_db=23.0, do_rms_norm=False, db_level=None, power=1.5, griffin_lim_iters=60, num_mels=80, mel_fmin=0.0, mel_fmax=None, spec_gain=20, do_amp_to_db_linear=True, do_amp_to_db_mel=True, pitch_fmax=640.0, pitch_fmin=1.0, signal_norm=True, min_level_db=-100, symmetric_norm=True, max_norm=4.0, clip_norm=True, stats_path=None)


In [3]:
config = Tacotron2Config(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    precompute_num_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    use_phonemes=True,
    phonemizer="gruut",
    phoneme_language="fr-fr",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache2"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    use_speaker_embedding=True,
    min_text_len=0,
    max_text_len=500,
    min_audio_len=100000,
    max_audio_len=400000,
)

In [12]:
print(config)

Tacotron2Config(output_path='/srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/french/cml_tts_dataset_french_v0.1', logger_uri=None, run_name='run', project_name=None, run_description='🐸Coqui trainer run.', print_step=25, plot_step=100, model_param_stats=False, wandb_entity=None, dashboard_logger='tensorboard', save_on_interrupt=True, log_model_step=10000, save_step=10000, save_n_checkpoints=5, save_checkpoints=True, save_all_best=False, save_best_after=10000, target_loss=None, print_eval=False, test_delay_epochs=-1, run_eval=True, run_eval_steps=None, distributed_backend='nccl', distributed_url='tcp://localhost:54321', mixed_precision=True, precision='fp16', epochs=100, batch_size=32, eval_batch_size=16, grad_clip=5.0, scheduler_after_epoch=True, lr=0.0001, optimizer='RAdam', optimizer_params={'betas': [0.9, 0.998], 'weight_decay': 1e-06}, lr_scheduler='NoamLR', lr_scheduler_params={'warmup_steps': 4000}, use_grad_scaler=False, allow_tf32=False, cudnn_enable=Tru

In [4]:
config.audio.fft_size = 2048
config.audio.win_length = 1200
config.audio.hop_length = 256
config.audio.num_mels = 80
config.audio.sample

In [11]:
!apt-get install espeak-ng

/var/lib/oar/.batch_job_bashrc: line 5: /home/ihernandez/.bashrc: No such file or directory
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [6]:
print(dataset_config)

BaseDatasetConfig(formatter='ljspeech2', dataset_name='', path='/srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/french/cml_tts_dataset_french_v0.1', meta_file_train='train.csv', ignored_speakers=None, language='fr', phonemizer='espeak', meta_file_val='', meta_file_attn_mask='')


In [13]:
ap = AudioProcessor.init_from_config(audio_config)

 > Setting up Audio Processor...
 | > sample_rate:24000
 | > resample:True
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:23.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [14]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

In [15]:
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 107598 files in /srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/french/cml_tts_dataset_french_v0.1


In [16]:
speaker_manager = SpeakerManager()
speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")
config.num_speakers = speaker_manager.num_speakers

model = Tacotron2(config, ap, tokenizer, speaker_manager=speaker_manager)


 > Init speaker_embedding layer.


In [17]:
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 32
 | > Num. of Torch Threads: 16
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/french/cml_tts_dataset_french_v0.1/run-November-01-2023_06+05PM-b417916

 > Model has 32649330 parameters


 > `speakers.pth` is saved to /srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/french/cml_tts_dataset_french_v0.1/run-November-01-2023_06+05PM-b417916/speakers.pth.
 > `speakers_file` is updated in the config.json.


In [18]:
trainer.fit()


 > EPOCH: 0/100
 --> /srv/storage/idmctal@storage1.nancy.grid5000.fr/2023/m2/adrelingyte/data/french/cml_tts_dataset_french_v0.1/run-November-01-2023_06+05PM-b417916




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: fr-fr
		| > phoneme backend: gruut
| > Number of instances : 106523



 > TRAINING (2023-11-01 18:05:45) 


 | > Preprocessing samples
 | > Max text length: 325
 | > Min text length: 8
 | > Avg text length: 141.72574867290118
 | 
 | > Max audio length: 399382.0
 | > Min audio length: 100821.0
 | > Avg audio length: 239262.54454204746
 | > Num. instances discarded samples: 21940
 | > Batch group size: 0.
ela!  tut ʁɛzɔ̃ də divɛʁʒɑ̃s avɛk ɛm.
 [!] Character '̃' not found in the vocabulary. Discarding it.
mɛ, tu ɑ̃ plɛzɑ̃tɑ̃t osi, lə pɛʁ ʁəfyzɛ.
 [!] Character '̃' not found in the vocabulary. Discarding it.
ʃnɛl fyt œʁø sɑ̃z elve la mwɛ̃dʁ ɔbʒɛksjɔ̃.
 [!] Character '̃' not found in the vocabulary. Discarding it.
ʒə nɔʁɛst sɛ pa si lə bʁamaputʁ okyp yn ɡʁɑ̃d plas dɑ̃ vo pɑ̃se.
 [!] Character '̃' not found in the vocabulary. Discarding it.
di lysjɛ̃.  —ʒe de tʁezɔʁ, ty i pɥizəʁa.
 [!] Character '—' not found in the vocabulary. Discarding it.
di lysjɛ̃.  —ʒe de tʁezɔʁ, ty i pɥizəʁa.
 [!] Character '—' not found in the vocabulary. Discarding it.
di lysjɛ̃.  —ʒe de tʁezɔʁ, ty i pɥizəʁa.
 [!] Charact

/home/ihernandez/.local/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

   --> TIME: 2023-11-01 18:16:19 -- STEP: 0/2644 -- GLOBAL_STEP: 0
     | > decoder_loss: 5.956602573394775  (5.956602573394775)
     | > postnet_loss: 7.947103500366211  (7.947103500366211)
     | > stopnet_loss: 0.5787264704704285  (0.5787264704704285)
     | > ga_loss: 0.007436789106577635  (0.007436789106577635)
     | > decoder_diff_spec_loss: 0.227020263671875  (0.227020263671875)
     | > postnet_diff_spec_loss: 4.239441394805908  (4.239441394805908)
     | > decoder_ssim_loss: 0.87890625  (0.87890625)
     | > postnet_ssim_loss: 0.7734375  (0.7734375)
     | > loss: 5.621537685394287  (5.621537685394287)
     | > align_error: 0.9820556640625  (0.9820556640625

Traceback (most recent call last):
  File "/home/ihernandez/.local/lib/python3.9/site-packages/trainer/trainer.py", line 1808, in fit
    self._fit()
  File "/home/ihernandez/.local/lib/python3.9/site-packages/trainer/trainer.py", line 1760, in _fit
    self.train_epoch()
  File "/home/ihernandez/.local/lib/python3.9/site-packages/trainer/trainer.py", line 1488, in train_epoch
    outputs, _ = self.train_step(batch, batch_num_steps, cur_step, loader_start_time)
  File "/home/ihernandez/.local/lib/python3.9/site-packages/trainer/trainer.py", line 1457, in train_step
    self.update_training_dashboard_logger(batch=batch, outputs=outputs)
  File "/home/ihernandez/.local/lib/python3.9/site-packages/trainer/utils/distributed.py", line 35, in wrapped_fn
    return fn(*args, **kwargs)
  File "/home/ihernandez/.local/lib/python3.9/site-packages/trainer/trainer.py", line 1950, in update_training_dashboard_logger
    self.model.train_log(
  File "/home/ihernandez/.local/lib/python3.9/site-packag

TypeError: object of type 'NoneType' has no len()

In [ ]:
!pip install trainer

In [ ]:
!pip install TTS

In [ ]:
!pip install TTS


In [ ]:
!pip install TTS

In [ ]:
ls -a ~

In [ ]:
!pip install huggingsound==0.1.6

In [10]:
!pip list

/var/lib/oar/.batch_job_bashrc: line 5: /home/ihernandez/.bashrc: No such file or directory
Package                   Version
------------------------- ----------
absl-py                   2.0.0
accelerate                0.24.1
aiohttp                   3.8.4
aiosignal                 1.3.1
anyascii                  0.3.2
appdirs                   1.4.4
async-timeout             4.0.2
attrs                     20.3.0
audioread                 3.0.0
Babel                     2.13.1
backcall                  0.2.0
bangla                    0.0.2
bcrypt                    3.1.7
beautifulsoup4            4.9.3
blinker                   1.6.3
bnnumerizer               0.0.2
bnunicodenormalizer       0.1.6
Bottleneck                1.2.1
cachetools                5.3.2
cephfs                    2.0.0
certifi                   2020.6.20
cffi                      1.14.5
chardet                   4.0.0
charset-normalizer        3.0.1
clean-fid                 0.1.35
click                     8.

In [ ]:
!pip install TTS
